In [1]:
from scipy.optimize import root, minimize
import numpy as np

In [2]:
C = [
 [9.9068, 7.4296,-5.3985,20.222],
 [8.8226,0.8404,1.8736,-21.6135],
#  [8.1358,0.0283,-0.1005,-21.509],
 [-1.7352,-4.7506,9.3576,5.6601]
]
T = 180+273.15
def lhK(T,i):
    C1 = 10**3
    C2 = 10**-2
    C3 = 10**-3
    term1 = C[i][0]*C1/T
    term2 = C[i][1]*C2*np.log(T)
    term3 = C[i][2]*C3*T
    term4 = C[i][3]
    return np.exp(term1+term2+term3+term4)
K = [lhK(T, i) for i in range(3)]
K

[2.5810870251924829e+17, 0.28834832096389584, 323.99999820141301]

In [4]:
x0 = {"CO2":0.0631,"NH3":0.5338,"H2NCOO":0,"NH4":0,"H2O":0.27,"HCO3":0,"H2NCONH2":0.1331}
def eqns(x):
    e1,e2,e3 = x
    T = 200+273.15
    K = [lhK(T, i) for i in range(3)]
    F0 = (1E6/60.06 - e3)/0.1331 #mol/day
    F = {"CO2":0.0631*F0 - e1 - e2,"NH3":0.5338*F0 - 2*e1 - e2,
         "H2NCOO":e1-e3,"NH4":e1+e2-e3,"H2O":0.27*F0 - e2 + e3,"HCO3":e2,"H2NCONH2":1E6/60.06}
    eqn1 = K[0]*((F["NH3"]**2)*F["CO2"]) - F["NH4"]*F["H2NCOO"]
    eqn2 = K[1]*(F["NH3"]*F["CO2"]*F["H2O"]) - F["NH4"]*F["HCO3"]
    eqn3 = K[2]*(F["NH4"]*F["H2NCOO"]) - F["H2O"]*F["H2NCONH2"]

    return eqn1,eqn2,eqn3

sol = root(eqns,[100,101,102],method="lm")

sol.x,eqns(sol.x)

(array([  5.68481139e+03,   7.85669098e-08,   4.65875840e+03]),
 (-26127264790362.148, -0.00011001002271872755, 3.6695272326469421))

In [5]:
def molar_fractions(x):
    e1,e2,e3 = x
    F0 = (1E6/60.06 - e3)/0.1331 #mol/day
    F = {"CO2":0.0631*F0 - e1 - e2,"NH3":0.5338*F0 - 2*e1 - e2,
         "H2NCOO":e1-e3,"NH4":e1+e2-e3,"H2O":0.27*F0 - e2 + e3,"HCO3":e2,"H2NCONH2":1E6/60.06}
    x = {}
    for key,value in F.items():
        x[key] = value/sum(F.values())
    return x

In [6]:
molar_fractions(sol.x)

{'CO2': -2.4882937866284262e-18,
 'NH3': 0.43505176646480098,
 'H2NCOO': 0.012155976451156748,
 'NH4': 0.012155976452087552,
 'H2O': 0.34337823210712681,
 'HCO3': 9.3080719616724929e-13,
 'H2NCONH2': 0.19725804852389703}